### Getting started: 4 steps to DeepCTR Estimator with TFRecord


### Step 1: Import model

In [1]:
import tensorflow as tf
from deepctr.estimator.inputs import input_fn_tfrecord
from deepctr.estimator.models import WDLEstimator


### Step 2: Generate feature columns for linear part and dnn part

In [2]:
sparse_features = ['C' + str(i) for i in range(1, 27)]
dense_features = ['I' + str(i) for i in range(1, 14)]

dnn_feature_columns = []
linear_feature_columns = []

In [3]:
for i, feat in enumerate(sparse_features):
    dnn_feature_columns.append(
        tf.feature_column.embedding_column(tf.feature_column.categorical_column_with_identity(feat, 1000), 4)
    )
    linear_feature_columns.append(tf.feature_column.categorical_column_with_identity(feat, 1000))
for feat in dense_features:
    dnn_feature_columns.append(tf.feature_column.numeric_column(feat))
    linear_feature_columns.append(tf.feature_column.numeric_column(feat))

In [4]:
print(dnn_feature_columns)

[EmbeddingColumn(categorical_column=IdentityCategoricalColumn(key='C1', number_buckets=1000, default_value=None), dimension=4, combiner='mean', initializer=<tensorflow.python.ops.init_ops.TruncatedNormal object at 0x7fe54573c6d0>, ckpt_to_load_from=None, tensor_name_in_ckpt=None, max_norm=None, trainable=True), EmbeddingColumn(categorical_column=IdentityCategoricalColumn(key='C2', number_buckets=1000, default_value=None), dimension=4, combiner='mean', initializer=<tensorflow.python.ops.init_ops.TruncatedNormal object at 0x7fe54573c710>, ckpt_to_load_from=None, tensor_name_in_ckpt=None, max_norm=None, trainable=True), EmbeddingColumn(categorical_column=IdentityCategoricalColumn(key='C3', number_buckets=1000, default_value=None), dimension=4, combiner='mean', initializer=<tensorflow.python.ops.init_ops.TruncatedNormal object at 0x7fe54573c790>, ckpt_to_load_from=None, tensor_name_in_ckpt=None, max_norm=None, trainable=True), EmbeddingColumn(categorical_column=IdentityCategoricalColumn(ke

### Step 3: Generate the training samples with TFRecord format

In [5]:
feature_description = {k: tf.io.FixedLenFeature(dtype=tf.int64, shape=1) for k in sparse_features}
feature_description.update(
    {k: tf.io.FixedLenFeature(dtype=tf.float32, shape=1) for k in dense_features}
)
feature_description['label'] = tf.io.FixedLenFeature(dtype=tf.float32, shape=1)

train_model_input = input_fn_tfrecord('./criteo_sample.tr.tfrecords', feature_description, 'label', batch_size=256,    num_epochs=1, shuffle_factor=10)
test_model_input  = input_fn_tfrecord('./criteo_sample.te.tfrecords', feature_description, 'label', batch_size=2 ** 14, num_epochs=1, shuffle_factor=0)

In [8]:
feature_description

{'C1': FixedLenFeature(shape=1, dtype=tf.int64, default_value=None),
 'C2': FixedLenFeature(shape=1, dtype=tf.int64, default_value=None),
 'C3': FixedLenFeature(shape=1, dtype=tf.int64, default_value=None),
 'C4': FixedLenFeature(shape=1, dtype=tf.int64, default_value=None),
 'C5': FixedLenFeature(shape=1, dtype=tf.int64, default_value=None),
 'C6': FixedLenFeature(shape=1, dtype=tf.int64, default_value=None),
 'C7': FixedLenFeature(shape=1, dtype=tf.int64, default_value=None),
 'C8': FixedLenFeature(shape=1, dtype=tf.int64, default_value=None),
 'C9': FixedLenFeature(shape=1, dtype=tf.int64, default_value=None),
 'C10': FixedLenFeature(shape=1, dtype=tf.int64, default_value=None),
 'C11': FixedLenFeature(shape=1, dtype=tf.int64, default_value=None),
 'C12': FixedLenFeature(shape=1, dtype=tf.int64, default_value=None),
 'C13': FixedLenFeature(shape=1, dtype=tf.int64, default_value=None),
 'C14': FixedLenFeature(shape=1, dtype=tf.int64, default_value=None),
 'C15': FixedLenFeature(shape

### Step 4: Train and evaluate the model

In [7]:
# model = DeepFMEstimator(linear_feature_columns, dnn_feature_columns, task='binary')
model = WDLEstimator(linear_feature_columns, dnn_feature_columns, dnn_hidden_units=[4, 4], dnn_dropout=0.5)

model.train(train_model_input)
eval_result = model.evaluate(test_model_input)

print(eval_result)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpi2b63iqo', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fe539ee4c10>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done c